In [1]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [2]:
#guardamos la ruta del archivo para posteriormente usar las solapas u hoja
archivo= 'datasets\homicidios.xlsx'

#Cargamos los solapas a utilizar en dataframes
df_homcidios_hechos = pd.read_excel(archivo, sheet_name='HECHOS')
df_hom_victimas = pd.read_excel(archivo,sheet_name='VICTIMAS')

In [3]:
#renombramos la columna ID_hecho por ID, para que coincida con la columna id de la solapa de Hechos
df_hom_victimas.rename(columns={'ID_hecho': 'ID'},inplace=True)

In [4]:
#Combinamos ambos df para obtener datos más completos 
df_homicidios = pd.merge(df_hom_victimas, df_homcidios_hechos, on='ID', how='inner')

In [5]:
df_homicidios.head(1)

,ID,FECHA_x,AAAA_x,MM_x,DD_x,ROL,VICTIMA_x,SEXO,EDAD,FECHA_FALLECIMIENTO,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA_y,ACUSADO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO


In [6]:
#eliminamos columnas duplicadas o que no nos serán de utilidad
columnas_drop= ['FECHA_x', 'AAAA_x','MM_x', 'DD_x','Altura', 'Cruce', 'Dirección Normalizada','AAAA_y','MM_y', 'DD_y']

df_homicidios.drop(columns= columnas_drop, inplace=True)

In [7]:
df_homicidios.head(1)

,ID,ROL,VICTIMA_x,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA_y,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA_y,ACUSADO
0,2016-0001,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00,1,2016-01-01,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO


In [8]:
#RENOMBRAMOS las siguientes columnas
df_homicidios.rename(columns={'VICTIMA_y':'VICTIMA', 'FECHA_y': 'FECHA'}, inplace=True)

In [9]:
#convertimos en mayusculas el primer carácter de cada palabra de las siguientes columnas y el resto en minusculas

columnas_convertir= ['ID','ROL','VICTIMA_x', 'VICTIMA','SEXO', 'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'XY (CABA)','pos x', 'pos y','PARTICIPANTES','ACUSADO']

#creamos un ciclo for para ahorrar código

for columna in columnas_convertir:
    df_homicidios[columna] = df_homicidios[columna].str.title()


In [10]:
#filtramos el df para mantener solo las filas donde las columnas no son iguales a 'Sd'
df_homicidios = df_homicidios[(df_homicidios['PARTICIPANTES'] != 'Sd') & (df_homicidios['VICTIMA'] != 'Sd') & (df_homicidios['ACUSADO'] != 'Sd')]

In [11]:
#función para eliminar y renombrar una columna
def clean_data(df_homicidios):
    df_homicidios.drop(columns=['VICTIMA'], inplace=True)
    df_homicidios.rename(columns={'VICTIMA_x': 'VICTIMA'}, inplace=True)
    return df_homicidios

df_homicidios =clean_data(df_homicidios.copy())

In [12]:
df_homicidios.head(10)

,ID,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,ACUSADO
0,2016-0001,Conductor,Moto,Masculino,19,2016-01-01 00:00:00,1,2016-01-01,04:00:00,4,Av Piedra Buena Y Av Fernandez De La Cruz,Avenida,Piedra Buena Av.,8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,Moto-Auto,Auto
1,2016-0002,Conductor,Auto,Masculino,70,2016-01-02 00:00:00,1,2016-01-02,01:15:00,1,Av Gral Paz Y Av De Los Corrales,Gral Paz,"Paz, Gral. Av.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,Auto-Pasajeros,Pasajeros
2,2016-0003,Conductor,Moto,Masculino,30,2016-01-03 00:00:00,1,2016-01-03,07:00:00,7,Av Entre Rios 2034,Avenida,Entre Rios Av.,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,Moto-Auto,Auto
4,2016-0005,Conductor,Moto,Masculino,29,2016-02-01 00:00:00,1,2016-01-21,05:20:00,5,Av San Juan Y Presidente Luis Saenz Peña,Avenida,San Juan Av.,1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,Moto-Pasajeros,Pasajeros
5,2016-0008,Conductor,Moto,Masculino,30,2016-01-24 00:00:00,1,2016-01-24,18:30:00,18,Av 27 De Febrero Y Av Escalada,Avenida,27 De Febrero Av.,8,Point (101721.59002217 93844.25656649),-58.44451316,-34.68475866,Moto-Objeto Fijo,Objeto Fijo
6,2016-0009,Pasajero_Acompañante,Moto,Masculino,29,2016-01-26 00:00:00,1,2016-01-24,19:10:00,19,Nogoya Y Joaquin V. Gonzales,Calle,Nogoya,11,Point (96545.87592078 102330.67262199),-58.50095869,-34.60825440,Moto-Auto,Auto
7,2016-0010,Conductor,Moto,Masculino,18,2016-01-29 00:00:00,1,2016-01-29,15:20:00,15,Av General Paz Y Av De Los Corrales,Gral Paz,"Paz, Gral. Av.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,Moto-Auto,Auto
8,2016-0012,Conductor,Moto,Masculino,22,2016-02-08 00:00:00,1,2016-02-08,01:20:00,1,Av Belgrano Y Bernardo De Irigoyen,Avenida,Belgrano Av.,1,Point (107595.35084333 101797.50052813),-58.38048577,-34.61303893,Moto-Cargas,Cargas
9,2016-0013,Peaton,Peaton,Masculino,16,2016-02-10 00:00:00,1,2016-02-10,11:30:00,11,Av Entre Rios 1366,Avenida,Entre Rios Av.,1,Point (106616.41069662 100496.44662323),-58.39114932,-34.62477387,Peaton-Auto,Auto
10,2016-0015,Peaton,Peaton,Femenino,16,2016-02-14 00:00:00,1,2016-02-14,05:14:00,5,Av Scalabrini Ortiz Y Vera,Avenida,"Scalabrini Ortiz, Raul Av.",15,Point (102357.43746828 103343.52002839),-58.43760020,-34.59912758,Peaton-Auto,Auto


In [13]:
#Relizaremos un cambio en el tipo de dato y llenamos los valores faltantes en la fecha de fallecimiento, suponiendo que falleció al momento del accidente vial, esto al desconocer la fecha exacta
df_homicidios['FECHA_FALLECIMIENTO']= df_homicidios['FECHA_FALLECIMIENTO'].astype(str)

df_homicidios['FECHA'] = df_homicidios['FECHA'].astype(str)

df_homicidios.loc[df_homicidios['FECHA_FALLECIMIENTO'] == 'SD', 'FECHA_FALLECIMIENTO'] = df_homicidios['FECHA']


In [14]:
#eliminamos el año del  inicio del ID
df_homicidios['ID'] = df_homicidios['ID'].str.split('-').str[1]

In [15]:
#comprobamos si la columna FECHA_FALLECIMIENTO contiene nulos para posteriormente reemplazarlos con los datos en FECHA,si es que estos se encuentran
df_homicidios['FECHA_FALLECIMIENTO'].isnull().sum()

0

In [16]:
#renombraremos la columna
df_homicidios.rename(columns={'FECHA': 'FECHA_SINIESTRO'}, inplace=True)

In [17]:
#Eliminamos la columna Calle ya que en LUGAR_DEL_HECHO encontramos los datos necesarios para el análisis
df_homicidios.drop(columns=['Calle'], inplace=True)

#también eliminamos XY(CABA) ya que contamos con columnas correspondientes para 'Y' y 'X'
df_homicidios.drop(columns=['XY (CABA)'], inplace=True)

In [18]:
#Cambiamos la columna 'HORA' por 'HORA_sieniestro'
df_homicidios.rename(columns={'HORA':'HORA_SINIESTRO'}, inplace=True)

In [19]:
# verificamos los duplicados

duplicates= df_homicidios.duplicated('ID')
duplicates

0      False
1      False
2      False
4      False
5      False
       ...  
712     True
713     True
714     True
715     True
716     True
Length: 690, dtype: bool

In [20]:
#reemplazamos duplicados por NaN
df_homicidios.loc[duplicates, 'ID'] = np.nan

In [21]:
#llenamos las columnas que contengan NaN con el índice correspondiente

df_homicidios['ID'] = df_homicidios['ID'].fillna(df_homicidios.index.to_series())

#convertimos la columna 'ID'a tipo entero
df_homicidios['ID'] = df_homicidios['ID'].astype(int)

In [22]:
duplicados= df_homicidios[df_homicidios.duplicated('ID')]['ID']
duplicados

99      99
146    146
147    147
150    150
153    153
154    154
156    156
157    157
160    160
163    163
164    164
165    165
168     30
169    169
172    172
173    173
175    175
176    176
177    177
178    178
273    149
275    151
282    158
Name: ID, dtype: int32

cambia el indice a los ID ponlos como el indice df_homicidios['ID'] = df_homicidios.index


In [23]:
#cómo aun hay duplicados en el id, se convertiran los id al valor del indice en el que se encuentren

df_homicidios['ID'] = df_homicidios.index

In [24]:
#volvemos a verificar si hay duplicados
duplicados= df_homicidios[df_homicidios.duplicated('ID')]['ID']
duplicados

Series([], Name: ID, dtype: int64)

In [25]:
df_homicidios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 690 entries, 0 to 716
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   690 non-null    int64 
 1   ROL                  690 non-null    object
 2   VICTIMA              690 non-null    object
 3   SEXO                 690 non-null    object
 4   EDAD                 690 non-null    object
 5   FECHA_FALLECIMIENTO  690 non-null    object
 6   N_VICTIMAS           690 non-null    int64 
 7   FECHA_SINIESTRO      690 non-null    object
 8   HORA_SINIESTRO       690 non-null    object
 9   HH                   690 non-null    object
 10  LUGAR_DEL_HECHO      690 non-null    object
 11  TIPO_DE_CALLE        690 non-null    object
 12  COMUNA               690 non-null    int64 
 13  pos x                690 non-null    object
 14  pos y                690 non-null    object
 15  PARTICIPANTES        690 non-null    object
 16  ACUSADO      

Se decidió completar los valores faltantes en la columna 'EDAD' con la media de todos los valores en dicha columna para completar los datos faltantes y de esa forma tener un análisis más preciso 

In [26]:
# convertir la columna 'edad' a numérica, ignorando los errores
df_homicidios['EDAD'] = pd.to_numeric(df_homicidios['EDAD'], errors='coerce')
#se calcula la media en la columna
media = df_homicidios['EDAD'].mean()
#reemplazamos 'Sd' por la media
df_homicidios['EDAD'] = df_homicidios['EDAD'].fillna(media)
#convertimos la columna a tipo entero
df_homicidios['EDAD'] = df_homicidios['EDAD'].astype(int)

In [27]:
#Las fechas del siniestro las integramos en semestres para el KPI
df_homicidios['FECHA_SINIESTRO'] = pd.to_datetime(df_homicidios['FECHA_SINIESTRO'])
df_homicidios['SEMESTRE'] = [1 if month <= 6 else 2 for month in df_homicidios['FECHA_SINIESTRO'].dt.month]

In [28]:
df_homicidios.tail()

,ID,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA_SINIESTRO,HORA_SINIESTRO,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,ACUSADO,SEMESTRE
712,712,Peaton,Peaton,Femenino,50,2021-12-12 00:00:00,1,2021-12-12,06:20:00,6,Av. Rivadavia Y Av. Pueyrredon,Avenida,3,-58.40596860,-34.61011987,Peaton-Auto,Auto,2
713,713,Pasajero_Acompañante,Moto,Femenino,18,2021-12-18 00:00:00,1,2021-12-13,17:10:00,17,Av. Riestra Y Mom,Avenida,7,-58.43353773,-34.64561636,Moto-Auto,Auto,2
714,714,Pasajero_Acompañante,Moto,Femenino,43,2021-12-20 00:00:00,1,2021-12-20,01:10:00,1,Au Dellepiane Y Lacarra,Autopista,9,-58.46739825,-34.65117757,Moto-Auto,Auto,2
715,715,Conductor,Moto,Masculino,27,2022-01-02 00:00:00,1,2021-12-30,00:43:00,0,Av. Gaona Y Terrada,Avenida,11,-58.47293407,-34.61984745,Moto-Cargas,Cargas,2
716,716,Conductor,Auto,Masculino,60,2021-12-20 00:00:00,1,2021-12-15,10:30:00,10,Av. Eva Peron 4071,Avenida,9,-58.47066794,-34.65021673,Auto-Cargas,Cargas,2


In [29]:
#hacemos un conteo de los 'Sd' en el df 
cont = df_homicidios.apply(lambda x: x.value_counts().get('Sd',0))
cont

ID                     0
ROL                    2
VICTIMA                0
SEXO                   2
EDAD                   0
FECHA_FALLECIMIENTO    0
N_VICTIMAS             0
FECHA_SINIESTRO        0
HORA_SINIESTRO         0
HH                     0
LUGAR_DEL_HECHO        0
TIPO_DE_CALLE          0
COMUNA                 0
pos x                  0
pos y                  0
PARTICIPANTES          0
ACUSADO                0
SEMESTRE               0
dtype: int64

In [30]:
#hacemos un conteo de los 'Sd' en el df 
cont = df_homicidios.apply(lambda x: x.value_counts().get('SD',0))
cont

ID                     0
ROL                    0
VICTIMA                0
SEXO                   0
EDAD                   0
FECHA_FALLECIMIENTO    0
N_VICTIMAS             0
FECHA_SINIESTRO        0
HORA_SINIESTRO         1
HH                     1
LUGAR_DEL_HECHO        0
TIPO_DE_CALLE          0
COMUNA                 0
pos x                  0
pos y                  0
PARTICIPANTES          0
ACUSADO                0
SEMESTRE               0
dtype: int64

In [31]:
#llenamos los faltantes en HORA
#extraemos las horas y minutos 
df_homicidios['HORA'] = df_homicidios['HORA_SINIESTRO'].str.split(':').str[0]
df_homicidios['MINUTOS'] = df_homicidios['HORA_SINIESTRO'].str.split(':').str[1]

# Convertir las horas y minutos a números
df_homicidios['HORA'] = pd.to_numeric(df_homicidios['HORA'], errors='coerce')
df_homicidios['MINUTOS'] = pd.to_numeric(df_homicidios['MINUTOS'], errors='coerce')

#calculamos la media de hrs y minutos que no son NaN
mean_hora = df_homicidios['HORA'].mean()
mean_min= df_homicidios['MINUTOS'].mean()

#Creamos la media de la hora como una cadena de texto
mean_tiempo = f"{int(mean_hora)}:{int(mean_min)}"

#reemplazamos 'Sd' con la media de la hora
df_homicidios.loc[df_homicidios['HORA_SINIESTRO'] == 'Sd', 'HORA_SINIESTRO'] = mean_tiempo
df_homicidios.loc[df_homicidios['HORA_SINIESTRO'] == 'SD', 'HORA_SINIESTRO'] = mean_tiempo

#Eliminar las columnas 'HORA' y 'MINUTOS'
df_homicidios = df_homicidios.drop(columns=['HORA', 'MINUTOS'])

In [32]:
df_homicidios.head()

,ID,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA_SINIESTRO,HORA_SINIESTRO,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,ACUSADO,SEMESTRE
0,0,Conductor,Moto,Masculino,19,2016-01-01 00:00:00,1,2016-01-01,04:00:00,4,Av Piedra Buena Y Av Fernandez De La Cruz,Avenida,8,-58.47533969,-34.68757022,Moto-Auto,Auto,1
1,1,Conductor,Auto,Masculino,70,2016-01-02 00:00:00,1,2016-01-02,01:15:00,1,Av Gral Paz Y Av De Los Corrales,Gral Paz,9,-58.50877521,-34.66977709,Auto-Pasajeros,Pasajeros,1
2,2,Conductor,Moto,Masculino,30,2016-01-03 00:00:00,1,2016-01-03,07:00:00,7,Av Entre Rios 2034,Avenida,1,-58.39040293,-34.63189362,Moto-Auto,Auto,1
4,4,Conductor,Moto,Masculino,29,2016-02-01 00:00:00,1,2016-01-21,05:20:00,5,Av San Juan Y Presidente Luis Saenz Peña,Avenida,1,-58.38718297,-34.62246630,Moto-Pasajeros,Pasajeros,1
5,5,Conductor,Moto,Masculino,30,2016-01-24 00:00:00,1,2016-01-24,18:30:00,18,Av 27 De Febrero Y Av Escalada,Avenida,8,-58.44451316,-34.68475866,Moto-Objeto Fijo,Objeto Fijo,1


In [33]:
#reorganizamos la dirección 
def rearganizar (address):
    words = address.split()
    if "Av." in  words:
        words.remove("Av.")
        words = ["Av."] + words
    return " ".join(words)

df_homicidios['LUGAR_DEL_HECHO'] = df_homicidios['LUGAR_DEL_HECHO'].apply(rearganizar)

In [34]:
formato_incorrecto = pd.to_datetime(df_homicidios['HORA_SINIESTRO'], errors='coerce').isnull()

#convertimos el formato ISO 8601 a HH:mm:ss
df_homicidios.loc[~formato_incorrecto, 'HORA_SINIESTRO'] = pd.to_datetime(df_homicidios.loc[~formato_incorrecto,'HORA_SINIESTRO']).dt.strftime('%H:%M:%S')

In [35]:
df_homicidios.loc[df_homicidios['HH'] == 'SD', 'HH'] = mean_tiempo

Eliminamos la columna HH ya que representa las horas en entero

In [36]:
df_homicidios = df_homicidios.drop(columns=['HH'])

In [37]:
df_homicidios.head()

,ID,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA_SINIESTRO,HORA_SINIESTRO,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,ACUSADO,SEMESTRE
0,0,Conductor,Moto,Masculino,19,2016-01-01 00:00:00,1,2016-01-01,04:00:00,Av Piedra Buena Y Av Fernandez De La Cruz,Avenida,8,-58.47533969,-34.68757022,Moto-Auto,Auto,1
1,1,Conductor,Auto,Masculino,70,2016-01-02 00:00:00,1,2016-01-02,01:15:00,Av Gral Paz Y Av De Los Corrales,Gral Paz,9,-58.50877521,-34.66977709,Auto-Pasajeros,Pasajeros,1
2,2,Conductor,Moto,Masculino,30,2016-01-03 00:00:00,1,2016-01-03,07:00:00,Av Entre Rios 2034,Avenida,1,-58.39040293,-34.63189362,Moto-Auto,Auto,1
4,4,Conductor,Moto,Masculino,29,2016-02-01 00:00:00,1,2016-01-21,05:20:00,Av San Juan Y Presidente Luis Saenz Peña,Avenida,1,-58.38718297,-34.62246630,Moto-Pasajeros,Pasajeros,1
5,5,Conductor,Moto,Masculino,30,2016-01-24 00:00:00,1,2016-01-24,18:30:00,Av 27 De Febrero Y Av Escalada,Avenida,8,-58.44451316,-34.68475866,Moto-Objeto Fijo,Objeto Fijo,1


In [39]:
#reemplazar valores '.' por 0 en las columnas 'pos x' y pos y'
df_homicidios['pos x'] = df_homicidios['pos x'].replace(".", 0)
df_homicidios['pos y'] = df_homicidios['pos y'].replace(".", 0)

In [40]:
#verificamos
df_homicidios.loc[(df_homicidios['pos x'] == 0) & (df_homicidios['pos y'] == 0)]

,ID,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA_SINIESTRO,HORA_SINIESTRO,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,ACUSADO,SEMESTRE
108,108,Conductor,Moto,Sd,42,2016-10-25,1,2016-10-25,00:00:00,Au Buenos Aires - La Plata Km. 4,Autopista,4,0,0,Moto-Cargas,Cargas,2
182,182,Conductor,Moto,Masculino,42,2017-04-10 00:00:00,1,2017-04-10,09:00:00,Av. Leopoldo Lugones Pkm 6900,Gral Paz,14,0,0,Moto-Cargas,Cargas,1
186,186,Conductor,Moto,Masculino,46,2017-04-28 00:00:00,2,2017-04-28,11:08:08,Au Perito Moreno Y Ramal Enlace Au1/Au6,Autopista,9,0,0,Moto-Cargas,Cargas,1
187,187,Pasajero_Acompañante,Moto,Masculino,16,2017-04-28 00:00:00,2,2017-04-28,11:08:08,Au Perito Moreno Y Ramal Enlace Au1/Au6,Autopista,9,0,0,Moto-Cargas,Cargas,1
188,188,Conductor,Auto,Masculino,33,2017-05-01 00:00:00,1,2017-05-01,03:47:47,Au Dellepiane 2400,Autopista,7,0,0,Auto-Auto,Auto,1
266,266,Conductor,Moto,Masculino,24,2017-11-19 00:00:00,1,2017-11-19,23:22:17,Au Arturo Frondizi Pkm 3100,Autopista,4,0,0,Moto-Pasajeros,Pasajeros,2
327,327,Peaton,Peaton,Masculino,37,2018-04-21 00:00:00,1,2018-04-21,22:15:00,Autopista Lugones Km 4.7,Autopista,14,0,0,Peaton-Auto,Auto,1
564,564,Conductor,Moto,Masculino,28,2020-05-17 00:00:00,1,2020-05-17,06:40:00,"Av. Lugones, Leopoldo Km 6,1",Autopista,14,0,0,Moto-Objeto Fijo,Objeto Fijo,1
578,578,Peaton,Peaton,Masculino,44,2020-09-01 00:00:00,1,2020-09-01,19:17:42,Murguiondo 2700,Calle,9,0,0,Peaton-Cargas,Cargas,2
642,642,Conductor,Moto,Masculino,24,2021-03-01 00:00:00,1,2021-03-01,09:20:00,"Au Buenos Aires La Plata Km 4,5",Autopista,4,0,0,Moto-Cargas,Cargas,1


In [41]:
#agregamos una columna para año mediante la FECHA_SINIESTRO
df_homicidios['Anio'] = df_homicidios['FECHA_SINIESTRO'].dt.year

In [42]:
df_homicidios.head()

,ID,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA_SINIESTRO,HORA_SINIESTRO,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,ACUSADO,SEMESTRE,Anio
0,0,Conductor,Moto,Masculino,19,2016-01-01 00:00:00,1,2016-01-01,04:00:00,Av Piedra Buena Y Av Fernandez De La Cruz,Avenida,8,-58.47533969,-34.68757022,Moto-Auto,Auto,1,2016
1,1,Conductor,Auto,Masculino,70,2016-01-02 00:00:00,1,2016-01-02,01:15:00,Av Gral Paz Y Av De Los Corrales,Gral Paz,9,-58.50877521,-34.66977709,Auto-Pasajeros,Pasajeros,1,2016
2,2,Conductor,Moto,Masculino,30,2016-01-03 00:00:00,1,2016-01-03,07:00:00,Av Entre Rios 2034,Avenida,1,-58.39040293,-34.63189362,Moto-Auto,Auto,1,2016
4,4,Conductor,Moto,Masculino,29,2016-02-01 00:00:00,1,2016-01-21,05:20:00,Av San Juan Y Presidente Luis Saenz Peña,Avenida,1,-58.38718297,-34.62246630,Moto-Pasajeros,Pasajeros,1,2016
5,5,Conductor,Moto,Masculino,30,2016-01-24 00:00:00,1,2016-01-24,18:30:00,Av 27 De Febrero Y Av Escalada,Avenida,8,-58.44451316,-34.68475866,Moto-Objeto Fijo,Objeto Fijo,1,2016


Agrupamos columnas necesarias para realizar el KPI 1

In [43]:
# agrupamos y sumamos el numero de victamas para cada semestre de cada año
df_homicidios_sum = df_homicidios.groupby(['Anio', 'SEMESTRE'])['N_VICTIMAS'].sum().reset_index()
df_homicidios_sum

,Anio,SEMESTRE,N_VICTIMAS
0,2016,1,62
1,2016,2,74
2,2017,1,77
3,2017,2,76
4,2018,1,76
5,2018,2,83
6,2019,1,56
7,2019,2,47
8,2020,1,31
9,2020,2,56


In [44]:
#definimos la tasa de accidentes por poblacion, KPI1
df_poblacion = pd.read_csv('Poblacion_Final.csv')

In [45]:
df_poblacion.head(1)

,Unnamed: 0,Comuna,Poblacion 2010,Crecimiento 2010-2022,Poblacion2016,Poblacion2017,Poblacion2018,Poblacion2019,Poblacion2020,Poblacion2021
0,0,13,231331,14.64,247686,250522,253391,256292,259227,262195


In [46]:
#eliminamos las columnas innecesarias
eliminar_col = ['Unnamed: 0', 'Poblacion 2010', 'Crecimiento 2010-2022']

df_poblacion.drop(columns=eliminar_col, inplace=True)
df_poblacion

,Comuna,Poblacion2016,Poblacion2017,Poblacion2018,Poblacion2019,Poblacion2020,Poblacion2021
0,13,247686,250522,253391,256292,259227,262195
1,14,237203,239129,241071,243028,245001,246990
2,12,217446,220476,223550,226666,229825,233028
3,4,224495,225554,226618,227687,228761,229840
4,1,214398,215850,217313,218785,220267,221760
5,7,218707,218395,218083,217772,217461,217151
6,8,195839,197311,198793,200287,201793,203309
7,6,189418,191738,194087,196464,198871,201307
8,11,196538,197678,198825,199979,201139,202306
9,15,190078,191359,192648,193946,195252,196567


In [47]:
#realizamos la suma de la población de cada año 
sum_poblacion = df_poblacion[['Poblacion2016', 'Poblacion2017', 'Poblacion2018', 'Poblacion2019', 'Poblacion2020', 'Poblacion2021']].sum()

In [48]:

sum_poblacion_df = sum_poblacion.to_frame().reset_index()
sum_poblacion_df.columns = ['Anio', 'Poblacion Total']

sum_poblacion_df

,Anio,Poblacion Total
0,Poblacion2016,3002178
1,Poblacion2017,3021444
2,Poblacion2018,3040889
3,Poblacion2019,3060508
4,Poblacion2020,3080306
5,Poblacion2021,3100282


In [49]:
#eliminamos la palabra poblacion en las filas
sum_poblacion_df['Anio'] = sum_poblacion_df['Anio'].str.replace('Poblacion', '', case=False)
sum_poblacion_df

,Anio,Poblacion Total
0,2016,3002178
1,2017,3021444
2,2018,3040889
3,2019,3060508
4,2020,3080306
5,2021,3100282


In [50]:
# modificamos el tipo de dato en la columna anio

df_homicidios_sum['Anio'] = df_homicidios_sum['Anio'].astype('int64')

In [51]:
df_homicidios_sum

,Anio,SEMESTRE,N_VICTIMAS
0,2016,1,62
1,2016,2,74
2,2017,1,77
3,2017,2,76
4,2018,1,76
5,2018,2,83
6,2019,1,56
7,2019,2,47
8,2020,1,31
9,2020,2,56


In [52]:
sum_poblacion_df['Anio'] = sum_poblacion_df['Anio'].astype('int64')
sum_poblacion_df.head(1)

,Anio,Poblacion Total
0,2016,3002178


In [53]:
df_hom_victimas = df_homicidios_sum.merge(sum_poblacion_df, on='Anio')

In [54]:
df_hom_victimas

,Anio,SEMESTRE,N_VICTIMAS,Poblacion Total
0,2016,1,62,3002178
1,2016,2,74,3002178
2,2017,1,77,3021444
3,2017,2,76,3021444
4,2018,1,76,3040889
5,2018,2,83,3040889
6,2019,1,56,3060508
7,2019,2,47,3060508
8,2020,1,31,3080306
9,2020,2,56,3080306


KPI 1 

Definimos a la tasa de homicidios en siniestros viales como el número de víctimas fatales en accidentes de tránsito por cada 100,000 habitantes en un área geográfica durante un período de tiempo específico. Su fórmula es: (Número de homicidios en siniestros viales / Población total) * 100,000

In [55]:
# (número de homicidios en siniestros viales / poblacion total) * 100000
df_hom_victimas['Tasa'] = (df_hom_victimas['N_VICTIMAS'] / df_hom_victimas['Poblacion Total']) * 100000
df_hom_victimas

,Anio,SEMESTRE,N_VICTIMAS,Poblacion Total,Tasa
0,2016,1,62,3002178,2.065167
1,2016,2,74,3002178,2.464877
2,2017,1,77,3021444,2.548450
3,2017,2,76,3021444,2.515354
4,2018,1,76,3040889,2.499269
5,2018,2,83,3040889,2.729465
6,2019,1,56,3060508,1.829762
7,2019,2,47,3060508,1.535693
8,2020,1,31,3080306,1.006394
9,2020,2,56,3080306,1.818001


In [56]:
#Obtenemos la tasa anterior 
df_hom_victimas['Tasa Anterior'] = df_hom_victimas['Tasa'].shift(periods=1, fill_value= 0)
df_hom_victimas

,Anio,SEMESTRE,N_VICTIMAS,Poblacion Total,Tasa,Tasa Anterior
0,2016,1,62,3002178,2.065167,0.000000
1,2016,2,74,3002178,2.464877,2.065167
2,2017,1,77,3021444,2.548450,2.464877
3,2017,2,76,3021444,2.515354,2.548450
4,2018,1,76,3040889,2.499269,2.515354
5,2018,2,83,3040889,2.729465,2.499269
6,2019,1,56,3060508,1.829762,2.729465
7,2019,2,47,3060508,1.535693,1.829762
8,2020,1,31,3080306,1.006394,1.535693
9,2020,2,56,3080306,1.818001,1.006394


In [57]:
#calculamos la variación porcentual entre la Tasa y la tasa Anterior y agregamos el resultado en La columna variacion tasa
df_hom_victimas['Variacion_Tasa'] = (df_hom_victimas['Tasa'] - df_hom_victimas['Tasa Anterior'])/ df_hom_victimas['Tasa Anterior'] * 100
df_hom_victimas

,Anio,SEMESTRE,N_VICTIMAS,Poblacion Total,Tasa,Tasa Anterior,Variacion_Tasa
0,2016,1,62,3002178,2.065167,0.000000,inf
1,2016,2,74,3002178,2.464877,2.065167,19.354839
2,2017,1,77,3021444,2.548450,2.464877,3.390562
3,2017,2,76,3021444,2.515354,2.548450,-1.298701
4,2018,1,76,3040889,2.499269,2.515354,-0.639451
5,2018,2,83,3040889,2.729465,2.499269,9.210526
6,2019,1,56,3060508,1.829762,2.729465,-32.962628
7,2019,2,47,3060508,1.535693,1.829762,-16.071429
8,2020,1,31,3080306,1.006394,1.535693,-34.466480
9,2020,2,56,3080306,1.818001,1.006394,80.645161


In [58]:
#cambiamos los valores infinitos por cero
df_hom_victimas['Variacion_Tasa'] = df_hom_victimas['Variacion_Tasa'].replace([np.inf, -np.inf], 0)


In [59]:
#redondeamos a 2 decimales la columna de Variación_Tasa
df_hom_victimas['Variacion_Tasa'] = df_hom_victimas['Variacion_Tasa'].round(2)
df_hom_victimas.head(2)

,Anio,SEMESTRE,N_VICTIMAS,Poblacion Total,Tasa,Tasa Anterior,Variacion_Tasa
0,2016,1,62,3002178,2.065167,0.000000,0.00
1,2016,2,74,3002178,2.464877,2.065167,19.35


KPI 2

Definimos a la cantidad de accidentes mortales de motociclistas en siniestros viales como el número absoluto de accidentes fatales en los que estuvieron involucradas víctimas que viajaban en moto en un determinado periodo temporal. Su fórmula para medir la evolución de los accidentes mortales con víctimas en moto es: (Número de accidentes mortales con víctimas en moto en el año anterior - Número de accidentes mortales con víctimas en moto en el año actual) / (Número de accidentes mortales con víctimas en moto en el año anterior) * 100

In [60]:
homicidios_moto = df_homicidios[df_homicidios['VICTIMA'] == 'Moto']
homicidios_moto

,ID,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,N_VICTIMAS,FECHA_SINIESTRO,HORA_SINIESTRO,LUGAR_DEL_HECHO,TIPO_DE_CALLE,COMUNA,pos x,pos y,PARTICIPANTES,ACUSADO,SEMESTRE,Anio
0,0,Conductor,Moto,Masculino,19,2016-01-01 00:00:00,1,2016-01-01,04:00:00,Av Piedra Buena Y Av Fernandez De La Cruz,Avenida,8,-58.47533969,-34.68757022,Moto-Auto,Auto,1,2016
2,2,Conductor,Moto,Masculino,30,2016-01-03 00:00:00,1,2016-01-03,07:00:00,Av Entre Rios 2034,Avenida,1,-58.39040293,-34.63189362,Moto-Auto,Auto,1,2016
4,4,Conductor,Moto,Masculino,29,2016-02-01 00:00:00,1,2016-01-21,05:20:00,Av San Juan Y Presidente Luis Saenz Peña,Avenida,1,-58.38718297,-34.62246630,Moto-Pasajeros,Pasajeros,1,2016
5,5,Conductor,Moto,Masculino,30,2016-01-24 00:00:00,1,2016-01-24,18:30:00,Av 27 De Febrero Y Av Escalada,Avenida,8,-58.44451316,-34.68475866,Moto-Objeto Fijo,Objeto Fijo,1,2016
6,6,Pasajero_Acompañante,Moto,Masculino,29,2016-01-26 00:00:00,1,2016-01-24,19:10:00,Nogoya Y Joaquin V. Gonzales,Calle,11,-58.50095869,-34.60825440,Moto-Auto,Auto,1,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,709,Conductor,Moto,Masculino,41,2021-12-11 00:00:00,1,2021-12-02,01:10:00,Av. Gaona 3655,Avenida,11,-58.47633683,-34.62140594,Moto-Auto,Auto,2,2021
711,711,Conductor,Moto,Masculino,24,2021-12-11 00:00:00,1,2021-12-11,23:00:00,Baigorria Y Victor Hugo,Calle,10,-58.51989389,-34.62284918,Moto-Auto,Auto,2,2021
713,713,Pasajero_Acompañante,Moto,Femenino,18,2021-12-18 00:00:00,1,2021-12-13,17:10:00,Av. Riestra Y Mom,Avenida,7,-58.43353773,-34.64561636,Moto-Auto,Auto,2,2021
714,714,Pasajero_Acompañante,Moto,Femenino,43,2021-12-20 00:00:00,1,2021-12-20,01:10:00,Au Dellepiane Y Lacarra,Autopista,9,-58.46739825,-34.65117757,Moto-Auto,Auto,2,2021


In [61]:
#agrupamos las columnas a utilizar y sumamos las victimas que hay por año
homicidios_moto = homicidios_moto.groupby(['Anio'])['N_VICTIMAS'].sum().reset_index()
homicidios_moto

,Anio,N_VICTIMAS
0,2016,63
1,2017,59
2,2018,61
3,2019,48
4,2020,32
5,2021,46


In [62]:
# hacemos una columna que represente el numero de victamas del año anterior al actual
homicidios_moto['N_Victimas_Anterior'] = homicidios_moto['N_VICTIMAS'].shift(periods=1, fill_value=0)
homicidios_moto

,Anio,N_VICTIMAS,N_Victimas_Anterior
0,2016,63,0
1,2017,59,63
2,2018,61,59
3,2019,48,61
4,2020,32,48
5,2021,46,32


In [63]:
# obtenemos la variación porcentual
homicidios_moto['Variacion_Victimas_Moto'] = (homicidios_moto['N_VICTIMAS'] -homicidios_moto['N_Victimas_Anterior']) / homicidios_moto['N_Victimas_Anterior'] * 100
homicidios_moto

,Anio,N_VICTIMAS,N_Victimas_Anterior,Variacion_Victimas_Moto
0,2016,63,0,inf
1,2017,59,63,-6.349206
2,2018,61,59,3.389831
3,2019,48,61,-21.311475
4,2020,32,48,-33.333333
5,2021,46,32,43.750000


In [64]:
#convertimos los infinitos en 0 
homicidios_moto['Variacion_Victimas_Moto'] = homicidios_moto['Variacion_Victimas_Moto'].replace([np.inf,-np.inf], 0)

In [65]:
#redondeamos a dos decimales
homicidios_moto['Variacion_Victimas_Moto'] = homicidios_moto['Variacion_Victimas_Moto'].round(2)
homicidios_moto.head(3)

,Anio,N_VICTIMAS,N_Victimas_Anterior,Variacion_Victimas_Moto
0,2016,63,0,0.00
1,2017,59,63,-6.35
2,2018,61,59,3.39


In [66]:
#procedemos a guardar los archivos en csv para su uso más adelante
homicidios_moto.to_csv('Tasa_Homicidios_Moto.csv', index=False)
df_hom_victimas.to_csv('Tasa_Homicidios.csv', index=False)
df_homicidios.to_csv('Homicidios_Final.csv', index=False)